This is a Jupyter notebook for quickly summarizing the status of a HOTOSM Tasking Manager campaing using the TM API.
It uses the same search functionality as the keyword search on tasks.hotosm.org/contribute

**Usage**:

1. Set your search keydword to `textSearch`
2. Execute all cells.
3. The results are sorted by project ID by default so the order stays consistent over time (bottom of the notebook)


In [1]:
# set the text search keyword
textSearch = 'covid'
instance = 'tasks.hotosm.org'

In [2]:

import requests
import json
import pandas as pd

# make sure pandas doesn't truncate the dataframe when rendered
pd.set_option('display.max_rows', 500)

def getAllPages(textSearch):
    page = 1
    while True:
        r = requests.get(f'https://tasks.hotosm.org/api/v1/project/search?textSearch={textSearch}&page={page}', 
            headers = {
                'Accept-Language': '*',
                'Content-Type': 'application/json'
            }
        )
        j = json.loads(r.content)
        df = pd.DataFrame(json.loads(r.content)['results'])
        yield df
        if j['pagination']['hasNext'] is True:
            page = j['pagination']['nextNum']
        else:
            break

            
# concatenate paginated API results into one dataframe
result = pd.concat(getAllPages(textSearch), ignore_index=True);
# search API pagination sometimes returns the same project more than once
result = result.drop_duplicates(subset='projectId');



In [3]:
# list of all fields for reference, for the sake
# of readability, some are not included in the final
# rendered table
print(list(result.columns))

['activeMappers', 'campaignTag', 'locale', 'mapperLevel', 'name', 'organisationTag', 'percentMapped', 'percentValidated', 'priority', 'projectId', 'shortDescription', 'status']


# Currently published projects

Drafts and archived project requires PM auth token.

Sorted by country, then project Id

In [6]:
# this cells applyies some sematic style to make the
# search summary nicer to read
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime

# colors from the HOTOSM media kit :)
hotosm_colors = {
  "red": "#D73F3F",
  "red-dark": "#6C2020",
  "red-light": "#FFEDED",
  "orange": "#FAA71E",
  "tan": "#F0EFEF",
  "blue-dark": "#2C3038",
  "blue-grey": "#68707F",
  "blue-light": "#929DB3",
  "grey-light": "#E1E0E0",
  "green": "#53AE62"
  }

# create a colormap for colorign mappedness and validatedness percentages
colors = [hotosm_colors['red'],hotosm_colors['orange'],hotosm_colors['green']]
colormap = LinearSegmentedColormap.from_list('hotosm-RdOrGr', colors, N=20)

# Pandas dataframe render styler functions
def highlight_priority(val):
    """set cell color based on proejct priority in TM"""
    lookup = {
        'LOW': hotosm_colors['grey-light'],
        'MEDIUM': hotosm_colors['green'],
        'HIGH': hotosm_colors['orange'],
        'URGENT': hotosm_colors['red']
    }
    color = lookup[val]
    return f'background-color: {color}'


# for notebooks used for reporting, it's nice to
# timestamp of the data, since it can get old quickly
now = datetime.utcnow()
print(f'last updated: {now} UTC')
# Total count of projects
print(f'total number of published projects: {len(result)}')

# We select only the most interesting fields from the dataframe
relevant = result[['projectId','name','percentMapped', 'percentValidated','priority','status', 'campaignTag']]
# Sorting by project id is somewhat cronological
# and also keeps the table order intact as new project are created
sorted_vals = relevant.sort_values('projectId')

# apply custom styler functions to final table
styled = sorted_vals.style\
    .background_gradient(cmap=colormap,subset=['percentMapped', 'percentValidated'])\
    .applymap(highlight_priority, subset=['priority'])

display(styled)

last updated: 2020-04-17 16:56:39.505670 UTC
total number of published projects: 56


,projectId,name,percentMapped,percentValidated,priority,status,campaignTag
55,5909,"COVID-19, Quehue, Anna, Cusco Peru",98,26,LOW,PUBLISHED,COVID-19
54,7372,"COVID-19 - Paucartambo 1, Cusco, Peru",99,6,LOW,PUBLISHED,COVID-19
53,7721,"COVID-19 - Coya - Calca, Peru",98,98,LOW,PUBLISHED,COVID-19
52,7768,"COVID-19 - Chamaca - Chumbivilcas, Peru",96,99,LOW,PUBLISHED,COVID-19
51,7770,"COVID-19 San Jeronimo - Cusco , Peru",97,59,LOW,PUBLISHED,COVID-19
5,7968,"COVID-19 - Andahuaylillas - Quispicanchi, Peru",95,56,HIGH,PUBLISHED,COVID-19
50,7970,"COVID-19 Lucre - Quispicanchi, Peru",100,100,LOW,PUBLISHED,COVID-19
49,8049,"COVID-19 Omacha - Paruro, Peru",98,59,LOW,PUBLISHED,COVID-19
48,8159,"Impendle, South Africa COVID-19",83,22,LOW,PUBLISHED,None
0,8188,Mapatón COVID-19 Medellín - Comuna 1,92,8,URGENT,PUBLISHED,None
